# This is a simple implementation of EfficientNet

In [2]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling2D

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


2024-11-30 21:02:17.673433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 21:02:17.742639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 21:02:17.762287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 21:02:17.897395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 21:02:19.096497: W tensorflow/compiler/tf2

# **MBConv Block: Mobile Inverted Bottleneck Convolution**

The **MBConv block** is a fundamental building block in lightweight neural networks like **EfficientNet** and **MobileNetV2**. It is designed to achieve high accuracy with minimal computational cost, leveraging techniques such as inverted bottlenecks, depthwise separable convolutions, and optional squeeze-and-excitation (SE) blocks.

---

## **Key Components of MBConv Block**

### **1. Expansion Phase**
- **Purpose:** Expands the input tensor's channel dimensions to a higher-dimensional space, enabling the model to capture richer features.
- **Operation:** A `1 × 1` pointwise convolution is applied to increase the channel size.


---

### **2. Depthwise Convolution**
- **Purpose:** Performs spatial filtering independently on each channel.
- **Operation:** A `k × k` depthwise convolution is applied, significantly reducing the computational cost compared to traditional convolutions.


Where:
- `C`: Number of input channels.
- `C'`: Number of output channels.
- `k`: Kernel size.

---

### **3. Squeeze-and-Excitation (SE) Block** *(Optional)*
- **Purpose:** Introduces channel-wise attention by recalibrating the importance of each channel.
- **Steps:**
  1. **Squeeze:** Apply global average pooling to compute channel-wise statistics.
  2. **Excite:** Use fully connected layers to compute channel weights, followed by a sigmoid activation.
  3. **Scale:** Multiply the original feature map by the learned weights.

---

### **4. Projection Phase**
- **Purpose:** Reduces the expanded feature map back to the desired number of output channels.
- **Operation:** A `1 × 1` pointwise convolution compresses the channel dimensions.

---

### **5. Residual Connection**
- **Purpose:** Adds the input to the output if:
  1. The stride is `1`.
  2. The number of input and output channels are the same.
- **Benefit:** Improves gradient flow and stability during training.


---

## **Diagram of MBConv Block**

Input
  |

[ Expand (1x1 Conv) ]  -- Expand phase
  |

[ Depthwise Conv (k x k) ] -- Spatial convolution
  |

[ SE Block (optional) ] -- Channel attention
  |

[ Projection (1x1 Conv) ] -- Reduce channels
  |

[ Residual Connection (if applicable) ]
  |
  
Output


In [11]:
def mbConv_block(input, input_channels, output_channel, t, s,kernel_size=3,se_ratio=0.25):
    """
    Constructs an MBConv block using the functional API with an SE block.

    Parameters:
    - input: Input tensor.
    - input_channels: Number of input channels.
    - output_channels: Number of output channels.
    - t: Expansion factor.
    - s: Stride for depthwise convolution.

    Returns:
    - Output tensor from the MBConv block.
    """
    # Expansion
    expanded_filters = input_channels * t

    # Expansion Convolution
    x = Conv2D(expanded_filters, 1, padding='same', use_bias=False)(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)

    # Depthwise Convolution
    x = Conv2D(filters=expanded_filters, kernel_size=kernel_size, padding='same', strides=s, use_bias=False, groups=expanded_filters)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)

    # Squeeze-and-Excitation (SE) Block
    # Squeeze-and-Excitation (SE) with configurable se_ratio
    se = GlobalAveragePooling2D()(x)
    se = tf.keras.layers.Reshape((1, 1, expanded_filters))(se)
    se = Conv2D(int(expanded_filters * se_ratio), kernel_size=1, activation="swish")(se)
    se = Conv2D(expanded_filters, kernel_size=1, activation="sigmoid")(se)
    x = tf.keras.layers.Multiply()([x, se])
    # Projection Convolution
    x = Conv2D(output_channel, kernel_size=1, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Skip connection (Residual connection)
    if s == 1 and input_channels == output_channel:
        x = tf.keras.layers.Add()([input, x])

    return x



# EfficientNet Architecture

EfficientNet is a family of convolutional neural networks (CNNs) that achieve state-of-the-art accuracy while being computationally efficient. The key innovation behind EfficientNet is the use of a compound scaling method that uniformly scales all dimensions of depth, width, and resolution using a set of fixed scaling coefficients.

## Key Components of EfficientNet

### 1. Compound Scaling
- **Purpose:** Efficiently scales the network to achieve better performance.
- **Method:** Uses a compound coefficient to uniformly scale network depth, width, and resolution.
- **Formula:** 
    \[
    \text{depth} = \alpha^d, \quad \text{width} = \beta^d, \quad \text{resolution} = \gamma^d
    \]
    where \( \alpha, \beta, \gamma \) are constants determined through a grid search, and \( d \) is the compound coefficient.

### 2. MBConv Blocks
- **Purpose:** Serve as the building blocks of EfficientNet, designed for efficiency and performance.
- **Components:** 
    - Expansion phase
    - Depthwise convolution
    - Squeeze-and-Excitation (SE) block (optional)
    - Projection phase
    - Residual connection (if applicable)

### 3. Squeeze-and-Excitation (SE) Blocks
- **Purpose:** Introduce channel-wise attention to recalibrate feature maps.
- **Steps:**
    1. **Squeeze:** Global average pooling to compute channel-wise statistics.
    2. **Excite:** Fully connected layers to compute channel weights, followed by a sigmoid activation.
    3. **Scale:** Multiply the original feature map by the learned weights.

### 4. Swish Activation Function
- **Purpose:** Improves model performance by providing a smooth, non-monotonic activation function.
- **Formula:** 
    \[
    \text{swish}(x) = x \cdot \text{sigmoid}(x)
    \]

## EfficientNet Variants
EfficientNet comes in several variants, each identified by a different scaling coefficient:
- **EfficientNet-B0:** Baseline model.
- **EfficientNet-B1 to B7:** Scaled versions of the baseline model, with increasing depth, width, and resolution.

## Advantages of EfficientNet
- **High Accuracy:** Achieves state-of-the-art performance on various benchmarks.
- **Computational Efficiency:** Requires fewer parameters and FLOPs compared to other models with similar accuracy.
- **Scalability:** The compound scaling method allows for easy scaling of the model to meet different resource constraints.

## Diagram of EfficientNet Architecture

```
Input
    |
[ Stem (Conv3x3) ]
    |
[ MBConv Block 1 ]
    |
[ MBConv Block 2 ]
    |
[ MBConv Block 3 ]
    |
[ MBConv Block 4 ]
    |
[ MBConv Block 5 ]
    |
[ MBConv Block 6 ]
    |
[ MBConv Block 7 ]
    |
[ Head (Conv1x1) ]
    |
[ Fully Connected Layer ]
    |
Output
```

In [12]:
def scaledResolution(phi):
    """
    Scaled Resolution function for EfficientNetB0-B7.
    """
    cases = {
        0: 224,
        1: 240,
        2: 260,
        3: 300,
        4: 380,
        5: 456,
        6: 528,
        7: 600
    }
    return cases[phi]

In [13]:
def efficientNet(input_shape=(224,224,3),num_classes=1000,phi=1):
    alpha = 1.2
    beta = 1.1
    gamma = 1.15

    depth = alpha ** phi
    width = beta ** phi
    scaled_resolution = scaledResolution(phi)
    scaled_input_shape = (scaled_resolution,scaled_resolution,input_shape[2])

    inputs = tf.keras.layers.Input(shape=scaled_input_shape)

    # Stem
    scaled_filters = int(32*width)
    x = Conv2D(scaled_filters,3,strides=2,padding='same',use_bias=False)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)

    # Block Configuration

    blocks_args = [
        (32,16,1,1,1,3), # Stage 1
        (16,24,6,2,int(1*depth),3), # Stage 2
        (24,40,6,2,int(2*depth),3), # Stage 3
        (40,80,6,2,int(2*depth),5), # Stage 4
        (80,112,6,1,int(3*depth),3), # Stage 5
        (112,192,6,2,int(3*depth),5), # Stage 6
        (192,320,6,1,int(4*depth),5), # Stage 7    
        (320,1280,6,1,int(1*depth),3) # Stage 8
    ]

    # MbConv Blocks
    for (input_channels,output_channels,t,s,repeats,kernel) in blocks_args:
        for i in range(repeats):
            x = mbConv_block(
                x,
                input_channels=input_channels if i == 0 else output_channels,
                output_channel=output_channels,
                t=t,
                s=s if i == 0 else 1,
                kernel_size=kernel
            )

    # Head
    scaled_filters = int(1280*width)
    x = Conv2D(scaled_filters,1,padding='same',use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(num_classes,activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs,outputs=x)

    return model

In [14]:
model = efficientNet(num_classes=10,phi=1)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 120, 120,  │        945 │ input_layer_2[0]… │
│                     │ 35)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        140 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 35)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 35)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 120, 120,  │      1,120 │ activation_6[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        128 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 120, 120,  │        288 │ activation_7[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        128 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ activation_8[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 32)  │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 1, 1, 8)   │        264 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 1, 1, 32)  │        288 │ conv2d_9[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 120, 120,  │          0 │ activation_8[0][… │
│ (Multiply)          │ 32)               │            │ conv2d_10[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 120, 120,  │        512 │ multiply_1[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │         64 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 120, 120,  │      1,536 │ batch_normalizat

 Total params: 21,593,323 (82.37 MB)

 Trainable params: 21,523,477 (82.11 MB)

 Non-trainable params: 69,846 (272.84 KB)

In [ ]:
def preprocess(data , labels, num_classes=10, target_shape=(240, 240)):
    
    data = tf.image.resize(data, target_shape)
    data = data/255.0
    labels = tf.squeeze(labels)  # Remove extra dimensions (e.g., from (32, 1) to (32,))
    labels = to_categorical(labels, num_classes)
    return data , labels

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
(x_val, y_val) = x_train[40000:], y_train[40000:]

x_train = x_train[:40000]
y_train = y_train[:40000]

# Create datasets and preprocess them
batch_size = 32
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(lambda x, y: preprocess(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .map(lambda x, y: preprocess(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    tf.data.Dataset.from_tensor_slices((x_val, y_val))
    .map(lambda x, y: preprocess(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Example: Iterate over the dataset
for batch_images, batch_labels in train_dataset.take(1):  # Process only one batch for demonstration
    print(f"Batch image shape: {batch_images.shape}")  # Expected: (batch_size, 240, 240, 3)
    print(f"Batch label shape: {batch_labels.shape}")  # Expected: (batch_size, 10)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5)

In [ ]:
efficientNetB0_model = efficientNetB0(num_classes=10,phi=1)


In [ ]:
efficientNetB0_model.summary()

In [ ]:
efficientNetB0_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = efficientNetB0_model.fit(train_dataset, validation_data=val_dataset, epochs=30)

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='validation accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='validation loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
def evaluate_model(model, test_dataset):
    loss, accuracy = model.evaluate(test_dataset)
    print(f"Test accuracy: {accuracy}")
    print(f"Test loss: {loss}")


In [ ]:
evaluate_model(efficientNetB0_model, test_dataset)

In [ ]:
def plot_predictions(model, dataset):
    plt.figure(figsize=(12, 12))
    classes = [
        "airplane", "automobile", "bird", "cat", "deer",
        "dog", "frog", "horse", "ship", "truck"
    ]
    for i, (images, labels) in enumerate(dataset.take(1)):
        pred = model.predict(images)
        for j in range(9):
            plt.subplot(3, 3, j + 1)
            plt.imshow(images[j])
            plt.title(f"Actual: {classes[labels[j].numpy().argmax()]}\nPredicted: {classes[pred[j].argmax()]}")
            plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_predictions(efficientNetB0_model, test_dataset)

In [ ]:
efficientNetB0_model.save('efficientNetB0_model.keras')